# Introduction to Brain Segmentation with Keras

# ***MAIN 2019 Educational Course***

## Thomas Funck

## McGill University

## **Contact**: email: [tffunck@gmail.com](mailto:tffunck@gmail.com) , Twitter: [@tffunck](https://twitter.com/tffunck)

# Load Code & Data

## Pull minc_keras from github

In [1]:
!rm -r /content/minc_keras
#Download repository
!git clone https://github.com/tfunck/minc_keras
  
#Switch dir
def set_base_dir():
  import os
  os.chdir('/content/minc_keras')
set_base_dir()


rm: cannot remove '/content/minc_keras': No such file or directory
Cloning into 'minc_keras'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 851 (delta 2), reused 0 (delta 0), pack-reused 845
Receiving objects: 100% (851/851), 79.93 MiB | 32.61 MiB/s, done.
Resolving deltas: 100% (450/450), done.


## Download and unzip data

In [2]:
#Download and unzip data
!tar -jxvf data/output.tar.bz2  &> /dev/null
!mv output mri

!wget https://amnesia.cbrain.mcgill.ca/deeplearning/sorteo.tar.bz2 --no-check-certificate
  
!mkdir -p pet
!tar -jxvf sorteo.tar.bz2 -C pet  &> /dev/null

--2018-12-10 18:23:15--  https://amnesia.cbrain.mcgill.ca/deeplearning/sorteo.tar.bz2
Resolving amnesia.cbrain.mcgill.ca (amnesia.cbrain.mcgill.ca)... 132.216.42.90
Connecting to amnesia.cbrain.mcgill.ca (amnesia.cbrain.mcgill.ca)|132.216.42.90|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 322192084 (307M) [application/x-bzip2]
Saving to: ‘sorteo.tar.bz2’

sorteo.tar.bz2      100%[===================>] 307.27M  85.8MB/s    in 3.8s    

2018-12-10 18:23:19 (80.9 MB/s) - ‘sorteo.tar.bz2’ saved [322192084/322192084]



## Initialize T1 MRI data

In [0]:
set_base_dir()
from utils import *
import numpy as np
from minc_keras import *

setup_dirs('mri_results')  
### Load data from brain images and save them into .npy. Sort them into train/validate/test splits
[images_mri, data_mri] = prepare_data('mri/', 'mri_results/data', 'mri_results/report', input_str='_T1w_anat_rsl', label_str='variant-seg',  clobber=False)

### 1) Load data
Y_validate_mri=np.load(data_mri["validate_y_fn"]+'.npy')
nlabels_mri=len(np.unique(Y_validate_mri))
X_train_mri=np.load(data_mri["train_x_fn"]+'.npy')
Y_train_mri=np.load(data_mri["train_y_fn"]+'.npy')
X_validate_mri=np.load(data_mri["validate_x_fn"]+'.npy')

X_test_mri=np.load(data_mri["test_x_fn"]+'.npy')
Y_test_mri=np.load(data_mri["test_y_fn"]+'.npy')

Y_test_mri=to_categorical(Y_test_mri)
Y_train_mri = to_categorical(Y_train_mri, num_classes=nlabels_mri)
Y_validate_mri = to_categorical(Y_validate_mri, num_classes=nlabels_mri)


1 [92, 110, 92]
2 [92, 110, 92]


## Initialize PET data

In [12]:
set_base_dir()
import minc_keras
from utils import *
import numpy as np
from minc_keras import *

setup_dirs('pet_results')  
### Load data from brain images and save them into .npy. Sort them into train/validate/test splits
[images_pet, data_pet] = prepare_data('pet/','pet_results/data','pet_results/report',input_str='_pet.mnc', label_str='brainmask', pad_base=3,ratios=[0.7,0.15], clobber=True)

### 1) Load data
Y_validate_pet=np.load(data_pet["validate_y_fn"]+'.npy')
nlabels_pet=len(np.unique(Y_validate_pet))
X_train_pet=np.load(data_pet["train_x_fn"]+'.npy')
Y_train_pet=np.load(data_pet["train_y_fn"]+'.npy')
X_validate_pet=np.load(data_pet["validate_x_fn"]+'.npy')

X_test_pet=np.load(data_pet["test_x_fn"]+'.npy')
Y_test_pet=np.load(data_pet["test_y_fn"]+'.npy')

Y_test_pet=to_categorical(Y_test_pet)
Y_train_pet = to_categorical(Y_train_pet, num_classes=nlabels_pet)
Y_validate_pet = to_categorical(Y_validate_pet, num_classes=nlabels_pet)


train : expected/real ratio = 70.00 / 73.33
validate : expected/real ratio = 15.00 / 20.00


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


1 [100, 100, 100]
2 [100, 104, 104]





# T1 Segmentation

* 261 MRI (skull stripped) 
 
* [1000 Functional Connectomes](http://fcon_1000.projects.nitrc.org/fcpClassic/FcpTable.html)

* GM/WM segmentation produced with FSL-5.0-fast

>>>>> __Input__

![alt text](https://github.com/tfunck/minc_keras/blob/master/images/t1_skullstripped.png?raw=true)

>>>>> __Label__

![alt text](https://github.com/tfunck/minc_keras/blob/master/images/gm_label.png?raw=true)







## Training a simple model

In [6]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from custom_loss import *
from keras.utils import to_categorical
from keras.activations import relu
from keras.layers.core import Dropout
from keras.callbacks import History, ModelCheckpoint

model_name="mri_1.hdf5"

IN = Input(shape=(data_mri["image_dim"][1], data_mri["image_dim"][2],1))
CONV1 = Conv2D(16, kernel_size=[3,3], activation="relu",padding='same')(IN)
CONV2 = Conv2D(16, kernel_size=[3,3], activation="relu",padding='same')(CONV1)
CONV3 = Conv2D(16, kernel_size=[3,3], activation="relu",padding='same')(CONV2)
OUT = Conv2D(nlabels_mri, kernel_size=[1,1], activation='softmax', padding='same')(CONV3)
model = keras.models.Model(inputs=[IN], outputs=OUT)
print(model.summary())
#set compiler
ada = keras.optimizers.Adam(0.0001)
#compile the model
model.compile(loss = 'binary_crossentropy', optimizer=ada,metrics=['acc'] )
#fit model
history = model.fit([X_train_mri],Y_train_mri, validation_data=([X_validate_mri], Y_validate_mri), epochs = 3)
#save model   
model.save(model_name)

test_score = model.evaluate(X_test_mri, Y_test_mri)
print("Test :", test_score)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 110, 92, 1)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 110, 92, 16)       160       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 110, 92, 16)       2320      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 110, 92, 16)       2320      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 110, 92, 3)        51        
Total params: 4,851
Trainable params: 4,851
Non-trainable params: 0
_________________________________________________________________
None
Train on 12539 samples, validate on 2391 samples
Epoch 1/3
12539/12539 [==============================] - 22s 2ms/step - loss: 0.4087 -

In [0]:
###Create predictions for model
from predict import *
predict('mri_1.hdf5', 'mri_results/predict/test', 'mri_results/data', 'mri_results/report/mri.csv', 'categorical_crossentropy', images_to_predict='1', category="test", verbose=True)

Model successfully loaded <keras.engine.training.Model object at 0x7f5d4bb25048>
Data loaded for prediction
label            mri/sub-58029/sub-58029_task-01_ses-01_T1w_var...
pet              mri/sub-58029/sub-58029_task-01_ses-01_T1w_ana...
subject                                                  sub-58029
task                                                             1
category                                                      test
valid_samples                                                   66
total_samples                                                   92
Name: 1, dtype: object
sub-58029_task-01_ses-01_T1w_anat_rsl.mnc 47 113
Saving prediction to: mri_results/predict/test//sub-58029_task-01_ses-01_T1w_anat_rsl_predict_1.png
Prediction completed


0

In [0]:
#If using Google Chrome, can download directly through browser
from google.colab import files
from glob import glob
for fn in glob('mri_results/predict/test/*.png') :  
  print(fn)
  files.download(fn)

#import matplotlib.pyplot as plt
#plt.imshow(plt.imread('mri_results/predict/test//sub-75922_task-01_ses-01_T1w_anat_rsl_predict_0.png'))

mri_results/predict/test/sub-75922_task-01_ses-01_T1w_anat_rsl_predict_0.png


## Adding drop-out

In [0]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from custom_loss import *
from keras.utils import to_categorical
from keras.activations import relu
from keras.layers.core import Dropout
from keras.callbacks import History, ModelCheckpoint

model_name="mri_2.hdf5"

IN = Input(shape=(data_mri["image_dim"][1], data_mri["image_dim"][2],1))
CONV1 = Conv2D(16, kernel_size=[3,3], activation="relu",padding='same')(IN)
DROPOUT1 = Dropout(0.2)(CONV1)
CONV2 = Conv2D(16, kernel_size=[3,3], activation="relu",padding='same')(DROPOUT1)
DROPOUT2 = Dropout(0.2)(CONV2)
CONV3 = Conv2D(16, kernel_size=[3,3], activation="relu",padding='same')(DROPOUT2)
DROPOUT3 = Dropout(0.2)(CONV3)
OUT = Conv2D(nlabels_mri, kernel_size=[1,1], activation='softmax', padding='same')(DROPOUT3)
model = keras.models.Model(inputs=[IN], outputs=OUT)

print(model.summary())

#set compiler
ada = keras.optimizers.Adam(0.0001)

#compile the model
model.compile(loss = 'binary_crossentropy', optimizer=ada,metrics=['acc'] )
#fit model

history = model.fit([X_train_mri],Y_train_mri,  validation_data=([X_validate_mri], Y_validate_mri), epochs = 3)
#save model   
model.save(model_name)

test_score = model.evaluate(X_test_mri, Y_test_mri)
print("Test :", test_score)


## Adding dilations

In [0]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from custom_loss import *
from keras.utils import to_categorical
from keras.activations import relu
from keras.layers.core import Dropout
from keras.callbacks import History, ModelCheckpoint

model_name="mri_3.hdf5"

IN = Input(shape=(data_mri["image_dim"][1], data_mri["image_dim"][2],1))
CONV1 = Conv2D(16, kernel_size=[3,3], dilation_rate=[2,2], activation="relu",padding='same')(IN)
CONV2 = Conv2D(16, kernel_size=[3,3], dilation_rate=[2,2], activation="relu",padding='same')(CONV1)
CONV3 = Conv2D(16, kernel_size=[3,3], dilation_rate=[2,2], activation="relu",padding='same')(CONV2)
OUT = Conv2D(nlabels_mri, kernel_size=[1,1], activation='softmax', padding='same')(CONV3)
model = keras.models.Model(inputs=[IN], outputs=OUT)

print(model.summary())

#set compiler
ada = keras.optimizers.Adam(0.0001)

#compile the model
model.compile(loss = 'binary_crossentropy', optimizer=ada,metrics=['acc'] )
#fit model

history = model.fit([X_train_mri],Y_train_mri,  validation_data=([X_validate_mri], Y_validate_mri), epochs = 3)
#save model   
model.save(model_name)

test_score = model.evaluate(X_test_mri, Y_test_mri)
print("Test :", test_score)


## __Exercises__

### 1. Modify the drop-out, dilation_rate, and number of kernels for one of the above templates. 

### 2.  Build a CNN with: 5 convolutional layers, 5x5 kernels and 8, 8, 16, 16, and 32 kernels in each layer

### 3. Train a CNN that has better than 0.95 test accuracy. How high can you get the accuracy without overfitting? 

## Solutions

### 1
Modify as you like! No wrong answers so long as it runs. 

### 2.

In [0]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from custom_loss import *
from keras.utils import to_categorical
from keras.activations import relu
from keras.layers.core import Dropout


model_name="mri_4.hdf5"

IN = Input(shape=(data_mri["image_dim"][1], data_mri["image_dim"][2],1))
CONV1 = Conv2D(8, kernel_size=[5,5], activation="relu",padding='same')(IN)
CONV2 = Conv2D(8, kernel_size=[5,5], activation="relu",padding='same')(CONV1)
CONV3 = Conv2D(16, kernel_size=[5,5], activation="relu",padding='same')(CONV2)
CONV4 = Conv2D(16, kernel_size=[5,5], activation="relu",padding='same')(CONV3)
CONV5 = Conv2D(32, kernel_size=[5,5], activation="relu",padding='same')(CONV4)
OUT = Conv2D(nlabels_mri, kernel_size=[1,1], activation='softmax', padding='same')(CONV5)
model = keras.models.Model(inputs=[IN], outputs=OUT)

print(model.summary())

#set compiler
ada = keras.optimizers.Adam(0.0001)

#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer=ada,metrics=['acc'] )
#fit model

history = model.fit([X_train_mri],Y_train_mri,  validation_data=([X_validate_mri], Y_validate_mri), epochs = 10)
#save model   
model.save(model_name)

test_score = model.evaluate(X_test_mri, Y_test_mri)
print("Test :", test_score)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 110, 92, 1)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 110, 92, 8)        208       
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 110, 92, 8)        1608      
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 110, 92, 16)       3216      
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 110, 92, 16)       6416      
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 110, 92, 32)       12832     
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 110, 92, 3)        99        
Total para

### 3.

Run the previous model for 10 iterations to exceed 0.950 test accuracy.

#U-Net

>>> __Input: Racploride PET__

![alt text](https://github.com/tfunck/minc_keras/blob/master/images/raclopride.png?raw=true)

>>> __Input: FDOPA PET__

![alt text](https://github.com/tfunck/minc_keras/blob/master/images/fdopa.png?raw=true)

>>> __Input: FDG PET__

![alt text](https://github.com/tfunck/minc_keras/blob/master/images/fdg.png?raw=true)

>>> __Input: Label__

![alt text](https://github.com/tfunck/minc_keras/blob/master/images/brainmask.png?raw=true)

## Building a U-NET in Keras

![](https://github.com/tfunck/minc_keras/blob/master/images/unet.png?raw=1)

Ronneberger, Fischer, and Brox. 2015."U-net: Convolutional networks for biomedical image segmentation." International Conference on Medical image computing and computer-assisted intervention. https://arxiv.org/abs/1505.04597

In [13]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from custom_loss import *
from keras.utils import to_categorical
from keras.activations import relu
from keras.layers.core import Dropout
from keras.layers import Input, Add, Multiply, Dense, BatchNormalization
from keras.layers import LeakyReLU, MaxPooling2D, Conv2DTranspose, Concatenate, ZeroPadding2D
from prepare_data import pad

### Warning : if you change the number of times you downsample with max_pool,
###           then you need to rerun prepare_data() with pad_base=<number of downsample nodes>
model_name="pet_1.hdf5"


### 1) Define architecture of neural network    
IN = Input(shape=(data_pet['image_dim'][1],data_pet['image_dim'][2] ,1))

BN1 = BatchNormalization()(IN)

conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(BN1)
conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
 
conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)

up6 = UpSampling2D(size=(2, 2))(conv4)
#up6 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv6)
conc6 = Concatenate(axis=3)([up6, conv3])
conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conc6)
conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

up7 = UpSampling2D(size=(2, 2))(conv7)
#up7 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv7)
conc7 = Concatenate(axis=3)([up7, conv2])
conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conc7) #(up8)
conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

up8 = UpSampling2D(size=(2, 2))(conv8)
#up8 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv8)
conc8 = Concatenate(axis=3)([up8, conv1])
conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conc8)
conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

conv10 = Convolution2D(nlabels_pet, 1, 1, activation='softmax')(conv9)

model = keras.models.Model(input=[IN], output=conv10)

print(model.summary())

#set compiler
ada = keras.optimizers.Adam(0.0001)

#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer=ada,metrics=['acc'] )
#fit model

history = model.fit([X_train_pet],Y_train_pet,  validation_data=([X_validate_pet], Y_validate_pet), epochs = 3)
#save model   
model.save(model_name)

test_score = model.evaluate(X_test_pet, Y_test_pet)
print("Test :", test_score)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 104, 104, 1)  0                                            
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 104, 104, 1)  4           input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 104, 104, 32) 320         batch_normalization_2[0][0]      
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 104, 104, 32) 9248        conv2d_16[0][0]                  
__________________________________________________________________________________________________
max_poolin

In [0]:
###Create predictions for model
from predict import *
predict(model_name, test_dir, data_dir, report_dir+'/pet.csv', 'categorical_crossentropy', images_to_predict='all', category="test", verbose=True)


In [0]:
#If using Google Chrome, can download directly through browser
#from google.colab import files
#from glob import glob
#for fn in glob('pet_results/predict/test/*.png') :  files.download(fn)

import matplotlib.pyplot as plt
plt.imshow(plt.imread('pet_results/predict/test//sub-D13_ses-01_task-01_acq-rcl_dwn-smpl_pet_predict_2.png'))


##Exercises

1. Modify the template above so that it has less than 120,000 parameters.

2. Use a transpose convolution to perform upsampling steps in the U-Net template. What happens to the number of parameters? How does accuracy change relative to this? 

3. Add another level of downsampling and up-sampling to the U-Net template. Remember to re-run the configuration cell with <pad_base=4> and <clobber=True> in order to pad the input images and labels appropriately relative to the number of times you use max pooling to downsample the images. 

4. Run a U-Net architecture on the GM-WM segementation task from part 1. How does the performance improvement compare to the increased number of parameters?

4. Using whatever techniques you like (downsampling, upsampling, dilations, drop-out, etc.), create the best architecture possible with least possible number of parameters. 

## Solutions


### 1.

In [15]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from custom_loss import *
from keras.utils import to_categorical
from keras.activations import relu
from keras.layers.core import Dropout
from keras.layers import Input, Add, Multiply, Dense, BatchNormalization
from keras.layers import LeakyReLU, MaxPooling2D, Conv2DTranspose, Concatenate, ZeroPadding2D
from prepare_data import pad

### Warning : if you change the number of times you downsample with max_pool,
###           then you need to rerun prepare_data() with pad_base=<number of downsample nodes>
model_name="mri_2.hdf5"


### 1) Define architecture of neural network    
IN = Input(shape=(data_pet['image_dim'][1],data_pet['image_dim'][2] ,1))

BN1 = BatchNormalization()(IN)

conv1 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(BN1)
conv1 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
 
conv2 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(pool1)
conv2 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(pool2)
conv3 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool3)
conv4 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv4)

up6 = UpSampling2D(size=(2, 2))(conv4)
#up6 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv6)
conc6 = Concatenate(axis=3)([up6, conv3])
conv7 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conc6)
conv7 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv7)

up7 = UpSampling2D(size=(2, 2))(conv7)
#up7 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv7)
conc7 = Concatenate(axis=3)([up7, conv2])
conv8 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(conc7) #(up8)
conv8 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(conv8)

up8 = UpSampling2D(size=(2, 2))(conv8)
#up8 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv8)
conc8 = Concatenate(axis=3)([up8, conv1])
conv9 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(conc8)
conv9 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(conv9)

conv10 = Convolution2D(nlabels_pet, 1, 1, activation='softmax')(conv9)

model = keras.models.Model(input=[IN], output=conv10)

print(model.summary())

#set compiler
ada = keras.optimizers.Adam(0.0001)

#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer=ada,metrics=['acc'] )
#fit model

history = model.fit([X_train_pet],Y_train_pet,  validation_data=([X_validate_pet], Y_validate_pet), epochs = 3)
#save model   
model.save(model_name)

test_score = model.evaluate(X_test_pet, Y_test_pet)
print("Test :", test_score)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` cal

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 104, 104, 1)  0                                            
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 104, 104, 1)  4           input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_46 (Conv2D)              (None, 104, 104, 8)  80          batch_normalization_4[0][0]      
__________________________________________________________________________________________________
conv2d_47 (Conv2D)              (None, 104, 104, 8)  584         conv2d_46[0][0]                  
__________________________________________________________________________________________________
max_poolin

### 2.

In [0]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from custom_loss import *
from keras.utils import to_categorical
from keras.activations import relu
from keras.layers.core import Dropout
from keras.layers import Input, Add, Multiply, Dense, BatchNormalization
from keras.layers import LeakyReLU, MaxPooling2D, Conv2DTranspose, Concatenate, ZeroPadding2D
from prepare_data import pad

### Warning : if you change the number of times you downsample with max_pool,
###           then you need to rerun prepare_data() with pad_base=<number of downsample nodes>
model_name="mri_3.hdf5"


### 1) Define architecture of neural network    
IN = Input(shape=(data['image_dim'][1],data['image_dim'][2] ,1))

BN1 = BatchNormalization()(IN)

conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(BN1)
conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
 
conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)

up6 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv4)
conc6 = Concatenate(axis=3)([up6, conv3])
conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conc6)
conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

up7 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv7)
conc7 = Concatenate(axis=3)([up7, conv2])
conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conc7) #(up8)
conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

up8 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv8)
conc8 = Concatenate(axis=3)([up8, conv1])
conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conc8)
conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

conv10 = Convolution2D(nlabels, 1, 1, activation='softmax')(conv9)

model = keras.models.Model(input=[IN], output=conv10)

print(model.summary())

#set compiler
ada = keras.optimizers.Adam(0.0001)

#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer=ada,metrics=['acc'] )
#fit model

history = model.fit([X_train],Y_train,  validation_data=([X_validate], Y_validate), epochs = 3)
#save model   
model.save(model_name)

test_score = model.evaluate(X_test, Y_test)
print("Test :", test_score)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           (None, 104, 104, 1)  0                                            
__________________________________________________________________________________________________
batch_normalization_5 (BatchNor (None, 104, 104, 1)  4           input_22[0][0]                   
__________________________________________________________________________________________________
conv2d_129 (Conv2D)             (None, 104, 104, 32) 320         batch_normalization_5[0][0]      
__________________________________________________________________________________________________
conv2d_130 (Conv2D)             (None, 104, 104, 32) 9248        conv2d_129[0][0]                 
__________________________________________________________________________________________________
max_poolin

### 3.

In [16]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from custom_loss import *
from keras.utils import to_categorical
from keras.activations import relu
from keras.layers.core import Dropout
from keras.layers import Input, Add, Multiply, Dense, BatchNormalization
from keras.layers import LeakyReLU, MaxPooling2D, Conv2DTranspose, Concatenate, ZeroPadding2D
from prepare_data import pad

set_base_dir()
import minc_keras
from utils import *
import numpy as np
from minc_keras import *

setup_dirs('pet_results')  
### Load data from brain images and save them into .npy. Sort them into train/validate/test splits
[images_pet, data_pet] = prepare_data('pet/','pet_results/data','pet_results/report',input_str='_pet.mnc', images_fn='pet.csv', label_str='brainmask', pad_base=3,ratios=[0.7,0.15], clobber=True)

### 1) Load data
Y_validate_pet=np.load(data_pet["validate_y_fn"]+'.npy')
nlabels_pet=len(np.unique(Y_validate_pet))
X_train_pet=np.load(data_pet["train_x_fn"]+'.npy')
Y_train_pet=np.load(data_pet["train_y_fn"]+'.npy')
X_validate_pet=np.load(data_pet["validate_x_fn"]+'.npy')

X_test_pet=np.load(data_pet["test_x_fn"]+'.npy')
Y_test_pet=np.load(data_pet["test_y_fn"]+'.npy')

Y_test_pet=to_categorical(Y_test_pet)
Y_train_pet = to_categorical(Y_train_pet, num_classes=nlabels_pet)
Y_validate_pet = to_categorical(Y_validate_pet, num_classes=nlabels_pet)


### Warning : if you change the number of times you downsample with max_pool,
###           then you need to rerun prepare_data() with pad_base=<number of downsample nodes>
model_name="mri_4.hdf5"


### 1) Define architecture of neural network    
IN = Input(shape=(data_pet['image_dim'][1],data_pet['image_dim'][2] ,1))

BN1 = BatchNormalization()(IN)

conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(BN1)
conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
 
conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

up5 = UpSampling2D(size=(2, 2))(conv5)
#up6 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv6)
conc5 = Concatenate(axis=3)([up5, conv4])
conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conc5)
conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

up6 = UpSampling2D(size=(2, 2))(conv6)
#up6 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv6)
conc6 = Concatenate(axis=3)([up6, conv3])
conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conc6)
conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

up7 = UpSampling2D(size=(2, 2))(conv7)
#up7 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv7)
conc7 = Concatenate(axis=3)([up7, conv2])
conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conc7) #(up8)
conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

up8 = UpSampling2D(size=(2, 2))(conv8)
#up8 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv8)
conc8 = Concatenate(axis=3)([up8, conv1])
conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conc8)
conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

conv10 = Convolution2D(nlabels_pet, 1, 1, activation='softmax')(conv9)

model = keras.models.Model(input=[IN], output=conv10)

print(model.summary())

#set compiler
ada = keras.optimizers.Adam(0.0001)

#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer=ada,metrics=['acc'] )
#fit model

history = model.fit([X_train_pet],Y_train_pet,  validation_data=([X_validate_pet], Y_validate_pet), epochs = 3)
#save model   
model.save(model_name)

test_score = model.evaluate(X_test_pet, Y_test_pet)
print("Test :", test_score)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` 

ValueError: ignored

### 4. 

In [0]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from custom_loss import *
from keras.utils import to_categorical
from keras.activations import relu
from keras.layers.core import Dropout
from keras.layers import Input, Add, Multiply, Dense, BatchNormalization
from keras.layers import LeakyReLU, MaxPooling2D, Conv2DTranspose, Concatenate, ZeroPadding2D
from prepare_data import pad

### Load data from brain images and save them into .npy. Sort them into train/validate/test splits
setup_dirs('mri_results')  
[images_mri_pad_4, data_mri_pad_4] = prepare_data('mri','mri_results/data', 'mri_results/report', input_str='_T1w_anat_rsl.mnc',\
                                                  label_str='variant-seg',images_fn='mri_unet.csv',pad_base=4, clobber=True)

### 1) Load data
Y_validate_mri_pad_4=np.load(data_mri_pad_4["validate_y_fn"]+'.npy')
nlabels_mri_pad_4=len(np.unique(Y_validate_mri_pad_4))

X_train_mri_pad_4=np.load(data_mri_pad_4["train_x_fn"]+'.npy')
Y_train_mri_pad_4=np.load(data_mri_pad_4["train_y_fn"]+'.npy')
X_validate_mri_pad_4=np.load(data_mri_pad_4["validate_x_fn"]+'.npy')

X_test_mri_pad_4=np.load(data_mri_pad_4["test_x_fn"]+'.npy')
Y_test_mri_pad_4=np.load(data_mri_pad_4["test_y_fn"]+'.npy')

Y_test_mri_pad_4=to_categorical(Y_test_mri_pad_4)
Y_train_mri_pad_4 = to_categorical(Y_train_mri_pad_4, num_classes=nlabels_mri_pad_4)
Y_validate_mri_pad_4 = to_categorical(Y_validate_mri_pad_4, num_classes=nlabels_mri_pad_4)

### Warning : if you change the number of times you downsample with max_pool,
###           then you need to rerun prepare_data() with pad_base=<number of downsample nodes>
model_name="mri_unet.hdf5"


### 1) Define architecture of neural network    
IN = Input(shape=(data_mri_pad_4['image_dim'][1],data_mri_pad_4['image_dim'][2] ,1))

BN1 = BatchNormalization()(IN)

conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(BN1)
conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
 
conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

up5 = UpSampling2D(size=(2, 2))(conv5)
#up6 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv6)
conc5 = Concatenate(axis=3)([up5, conv4])
conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conc5)
conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

up6 = UpSampling2D(size=(2, 2))(conv6)
#up6 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv6)
conc6 = Concatenate(axis=3)([up6, conv3])
conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conc6)
conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

up7 = UpSampling2D(size=(2, 2))(conv7)
#up7 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv7)
conc7 = Concatenate(axis=3)([up7, conv2])
conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conc7) #(up8)
conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

up8 = UpSampling2D(size=(2, 2))(conv8)
#up8 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv8)
conc8 = Concatenate(axis=3)([up8, conv1])
conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conc8)
conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

conv10 = Convolution2D(nlabels_mri_pad_4, 1, 1, activation='softmax')(conv9)

model = keras.models.Model(input=[IN], output=conv10)

print(model.summary())

#set compiler
ada = keras.optimizers.Adam(0.0001)
#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer=ada,metrics=['acc'] )
#fit model
history = model.fit([X_train_mri_pad_4],Y_train_mri_pad_4,  validation_data=([X_validate_mri_pad_4], Y_validate_mri_pad_4), epochs = 3)
#save model   
model.save(model_name)

test_score = model.evaluate(X_test_mri_pad_4, Y_test_mri_pad_4)
print("Test :", test_score)


In [0]:
###Create predictions for model
from predict import *
predict('mri_unet.hdf5', test_dir, 'mri_results/data', 'mri_results/report/mri_unet.csv', 'categorical_crossentropy', images_to_predict='all', category="test", verbose=True)

In [27]:
from google.colab import files
from glob import glob
for fn in glob('pet_results/predict/test/*.png') :  
  print(fn)
  files.download(fn)

pet_results/predict/test/sub-70106_task-01_ses-01_T1w_anat_rsl_predict_0.png
pet_results/predict/test/sub-55652_task-01_ses-01_T1w_anat_rsl_predict_2.png
pet_results/predict/test/sub-06880_task-01_ses-01_T1w_anat_rsl_predict_1.png


### 5.

Modify the code for solutions 3 to improve network for PET segmentation.

Modify answer to question 4 to improve MRI segmentation.